In [1]:
! pip install pandas
! pip install boto3
! pip install watchtower
! pip install s3fs==0.4.2
! pip install pyathena
! pip install matplotlib
! pip install scipy
! pip install ipywidgets
! pip install scikit-learn

! conda install -c conda-forge --yes implicit 

! pip install --upgrade jupyter_client # useful to make ipywidgets work properly when fitting data with implicit

# pip install git+https://gitlab.com/cloena/cloena-aws-tools.git

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrad

In [2]:
import pandas as pd

import implicit

from aws_tools import athena_tools, s3_tools

import matplotlib

import re

import scipy

from ipywidgets import FloatProgress

import numpy as np

from sklearn.preprocessing import MinMaxScaler

from datetime import datetime
import json

from io import StringIO # python3; python2: BytesIO 
import boto3
import csv

from pyathena import connect

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/aws_tools/project_config.py:18: UserWarning: no aws_config.json file found
  warnings.warn('no aws_config.json file found')


In [52]:
# create a connection to Athena
conn = connect(s3_staging_dir = 's3://aws-athena-query-results-341377015103-eu-west-2/',
                   region_name='eu-west-2') 

### Assemble the 'views' data

In [53]:
query = '''
DROP TABLE IF EXISTS bt_home_ds.bt_tv_recommendation_engine_data_views;
'''
pd.read_sql(query, conn)

query = '''
CREATE TABLE IF NOT EXISTS bt_home_ds.bt_tv_recommendation_engine_data_views AS
SELECT
    vs.VISION_SERVICE_ID AS id_user -- account ID
    ,vs.CI_ASSET_TYPE AS TYPE_ASSET -- e.g. Film, Music, etc.
    ,vs.VIEW_TIME_ST AS EVENT_DATE -- time of event
    
    ,COALESCE(ps2.EDITORIAL_VERSION_ID, ps1.EDITORIAL_VERSION_ID) AS id_editorial -- identifies multiple instances of same film/season/episode, e.g. rent/purchase & SD/HD
    
    ,COALESCE(ps2.CI_TITLE, ps1.CI_TITLE) AS TITLE --  human readable title
    ,COALESCE(ps2.CI_TYPE, ps1.CI_TYPE) AS TYPE -- type, like film/music/episode/season/collection
    ,COALESCE(ps2.CI_AVAILABLE_END_DT, ps1.CI_AVAILABLE_END_DT) AS END_DATE -- date until availability of item
    ,COALESCE(ps2.GENRE, ps1.GENRE) AS GENRE
    ,COALESCE(ps2.rating, ps1.rating) AS rating
      
    ,'VIEW' AS type_entitlement
    
FROM
    bt_home_datamart.l_edw_vod_views vs

-- perform an inner join of the events with the catalogue to get the product id for each entry:
-- this is needed to later match the product if with the parent id and group episodes into seasons
INNER JOIN 
    bt_home_datamart.l_edw_vod_products ps1
ON
     vs.CONTENT_ID = ps1.PRODUCT_GUID

-- perform left join to match ID_PARENT with product id where available:
-- this will give either the same element due to the coalesce OR the actual parent, i.e. the season rather than the episode
LEFT JOIN 
   bt_home_datamart.l_edw_vod_products ps2
ON
   ps1.CI_PARENTGUID = ps2.PRODUCT_GUID 

WHERE vs.EVENT_SLOT_TYPE = 'Feature'
AND vs.VISION_SERVICE_ID IS NOT NULL
AND ps1.EDITORIAL_VERSION_ID IS NOT NULL;
'''
pd.read_sql(query, conn)

,rows


In [54]:
query = '''
DROP TABLE IF EXISTS bt_home_ds.bt_tv_recommendation_engine_data_views_maxenddt;
'''
pd.read_sql(query, conn)

query = '''
CREATE TABLE IF NOT EXISTS bt_home_ds.bt_tv_recommendation_engine_data_views_maxenddt AS
SELECT 
    aaa.id_user -- account ID
    ,aaa.TYPE_ASSET -- e.g. Film, Music, etc.
    ,aaa.EVENT_DATE -- time of event
    
    ,aaa.id_editorial -- identifies multiple instances of same film/season/episode, e.g. rent/purchase & SD/HD
    
    ,aaa.TITLE --  human readable title
    ,aaa.TYPE -- type, like film/music/episode/season/collection
    
    ,COALESCE(ps.CI_AVAILABLE_END_DT, aaa.END_DATE) AS END_DATE -- date until availability of item
    
    ,aaa.GENRE
    ,aaa.rating
    
    ,aaa.type_entitlement -- TVOD / EVOD / PPV

FROM bt_home_ds.bt_tv_recommendation_engine_data_views aaa

-- left join with catalogue again where we keep only max availability date for each editorial id
LEFT JOIN
    (SELECT EDITORIAL_VERSION_ID, MAX(DATE(CI_AVAILABLE_END_DT)) AS CI_AVAILABLE_END_DT
    FROM bt_home_datamart.l_edw_vod_products 
    GROUP BY EDITORIAL_VERSION_ID) ps
ON 
    aaa.id_editorial = ps.EDITORIAL_VERSION_ID;
'''
pd.read_sql(query, conn)

,rows


In [ ]:
query = """
SELECT *
FROM bt_home_ds.bt_tv_recommendation_engine_data_views_maxenddt;
"""

data_views = athena_tools.AthenaQuerier().execute_query(query=query)

2020-12-08 13:35:59,617 [INFO ]  starting Athena query ...


### Assemble the 'purchase'/'rental'/'PPV' data

In [ ]:
query = '''
DROP TABLE IF EXISTS bt_home_ds.bt_tv_recommendation_engine_data_prps;
'''
pd.read_sql(query, conn)

query = '''
CREATE TABLE IF NOT EXISTS bt_home_ds.bt_tv_recommendation_engine_data_prps AS
SELECT
    vs.VISION_SERVICE_ID AS id_user -- account ID
    ,vs.CI_ASSET_TYPE AS TYPE_ASSET -- e.g. Film, Music, etc.
    ,vs.PURCHASE_TIME_ST AS EVENT_DATE -- time of event
    
    ,COALESCE(ps2.EDITORIAL_VERSION_ID, ps1.EDITORIAL_VERSION_ID) AS id_editorial -- identifies multiple instances of same film/season/episode, e.g. rent/purchase & SD/HD
    
    ,COALESCE(ps2.CI_TITLE, ps1.CI_TITLE) AS TITLE --  human readable title
    ,COALESCE(ps2.CI_TYPE, ps1.CI_TYPE) AS TYPE -- type, like film/music/episode/season/collection
    
    ,COALESCE(ps3.CI_AVAILABLE_END_DT, ps2.CI_AVAILABLE_END_DT, ps1.CI_AVAILABLE_END_DT) AS END_DATE -- date until availability of item
    
    ,COALESCE(ps2.GENRE, ps1.GENRE) AS GENRE
    ,COALESCE(ps2.rating, ps1.rating) AS rating
    
    ,vs.ENTITLEMENT_TYPE AS type_entitlement -- TVOD / EVOD / PPV
    
FROM
    bt_home_datamart.l_edw_vod_purchases vs

-- perform an inner join of the events with the catalogue to get the product id for each entry:
-- this is needed to later match the product if with the parent id and group episodes into seasons
INNER JOIN 
    bt_home_datamart.l_edw_vod_products ps1
ON
    vs.PRODUCT_ID = ps1.PRODUCT_GUID

-- perform left join to match ID_PARENT with product id where available:
-- this will give either the same element due to the coalesce OR the actual parent, i.e. the season rather than the episode
LEFT JOIN 
   bt_home_datamart.l_edw_vod_products ps2
ON
   ps1.CI_PARENTGUID = ps2.PRODUCT_GUID 
   
-- left join with catalogue again where we keep only max availability date for each editorial id
LEFT JOIN
    (SELECT EDITORIAL_VERSION_ID, MAX(DATE(CI_AVAILABLE_END_DT)) AS CI_AVAILABLE_END_DT
    FROM bt_home_datamart.l_edw_vod_products 
    GROUP BY EDITORIAL_VERSION_ID) ps3
ON 
    id_editorial = ps3.EDITORIAL_VERSION_ID

WHERE vs.VISION_SERVICE_ID IS NOT NULL
AND ps2.EDITORIAL_VERSION_ID IS NOT NULL;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
DROP TABLE IF EXISTS bt_home_ds.bt_tv_recommendation_engine_data_prps_maxenddt;
'''
pd.read_sql(query, conn)

query = '''
CREATE TABLE IF NOT EXISTS bt_home_ds.bt_tv_recommendation_engine_data_prps_maxenddt AS
SELECT
    aaa.id_user -- account ID
    ,aaa.TYPE_ASSET -- e.g. Film, Music, etc.
    ,aaa.EVENT_DATE -- time of event
    
    ,aaa.id_editorial -- identifies multiple instances of same film/season/episode, e.g. rent/purchase & SD/HD
    
    ,aaa.TITLE --  human readable title
    ,aaa.TYPE -- type, like film/music/episode/season/collection
    
    ,COALESCE(ps.CI_AVAILABLE_END_DT, aaa.END_DATE) AS END_DATE -- date until availability of item
    
    ,aaa.GENRE
    ,aaa.rating
    
    ,aaa.type_entitlement -- TVOD / EVOD / PPV
    
FROM
    bt_home_ds.bt_tv_recommendation_engine_data_prps aaa

-- left join with catalogue again where we keep only max availability date for each editorial id
LEFT JOIN
    (SELECT EDITORIAL_VERSION_ID, MAX(DATE(CI_AVAILABLE_END_DT)) AS CI_AVAILABLE_END_DT
    FROM bt_home_datamart.l_edw_vod_products 
    GROUP BY EDITORIAL_VERSION_ID) ps
ON 
    aaa.id_editorial = ps.EDITORIAL_VERSION_ID;
'''
pd.read_sql(query, conn)

In [ ]:
query = """
SELECT *
FROM bt_home_ds.bt_tv_recommendation_engine_data_prps_maxenddt;
"""

data_prps = athena_tools.AthenaQuerier().execute_query(query=query)

### Model Data Prep

In [ ]:
data_implicit = data_views.append(data_prps, ignore_index=True) 

In [ ]:
del data_views
del data_prps

In [ ]:
data_implicit.head(1)

In [ ]:
data_implicit['type_asset'].unique()

In [ ]:
DROP_MUSIC = True
if DROP_MUSIC:
    data_implicit = data_implicit[(data_implicit['type_asset'] != 'Music') & (data_implicit['type_asset'] != 'music')]
else:
    pass

In [ ]:
# Richard's export from the hackathon
# season_to_brand_map = pd.read_csv('s3://bt-data-science-playground/bt-tv-recommendation-system/model_objects/SeasonToBrandMapping-NoExpired.csv')

In [ ]:
season_to_brand_map = pd.read_csv('s3://bt-data-science-playground/bt-tv-recommendation-system/model_objects/New-SeasonToBrandMapping.csv')

In [ ]:
season_to_brand_map.head() 

In [ ]:
def season_to_brand(df, df_map):
    df = df.merge(df_map, how='left', left_on = 'id_editorial', right_on = 'SEASON_GUID')
    df['BRAND_GUID'].fillna(df['id_editorial'], inplace=True)
    df['id_editorial'] = df['BRAND_GUID']
    df.drop(['SEASON_GUID', 'BRAND_GUID'], axis=1, inplace=True)
    
    return df

In [ ]:
data_implicit = season_to_brand(data_implicit, season_to_brand_map)

In [ ]:
# Calculate sparsity 
n_users = data_implicit['id_user'].unique().shape[0]
n_items = data_implicit['id_editorial'].unique().shape[0]

print('Number of users: {}'.format(n_users))
print('Number of items: {}'.format(n_items))
print('Sparsity: {:4.3f}%'.format(float(data_implicit.shape[0]) / float(n_users*n_items) * 100))

In [ ]:
# Calculate how many users have watched X% of the content
idx = data_implicit['id_user'].value_counts().index.tolist()
counts = data_implicit['id_user'].value_counts().tolist()

user_count = []
percentage_activity = []
activity_so_far = 0
total_activity = sum(counts)

for u in range(len(idx)):
    
    user_count.append(u+1)
    
    activity_so_far = activity_so_far + counts[u]
    percentage_so_far = activity_so_far / total_activity * 100
    percentage_activity.append(percentage_so_far)
     
import matplotlib.pyplot as plt

plt.plot(user_count, percentage_activity)
plt.xlabel('User count')
plt.ylabel('Total # of activity [%]')
plt.grid()
plt.yticks(np.arange(0,105, 5))
plt.xticks(np.arange(0,1000000, 150000))
plt.show()

In [ ]:
data_implicit['type_entitlement'].value_counts()

In [ ]:
# based on type of entitlement (purchase/rental/ppv/view) give different strength [EVOD=purchase;TVOD=rental]
def assign_eventStrength(x):
    if x == 'EVOD':
        val = 3
    elif x == 'TVOD':
        val = 3
    elif x == 'PPV':
        val = 3
    elif x == 'VIEW':
        val = 1
    else:
        val = 1
    return val

data_implicit['eventStrength'] = data_implicit['type_entitlement'].apply(assign_eventStrength)

In [ ]:
# Turn duplicate the rating column in a form that is numeric so that they can be compared
def rating_toNumeric(x):
    if x == 'u':
        return 0
    elif x == 'pg':
        return 1
    elif x == '12':
        return 2
    elif x == '15':
        return 3
    elif x == '18':
        return 4
    else:
        return 5
    
data_implicit['rating_n'] = data_implicit['rating'].apply(rating_toNumeric)

In [ ]:
def type_production(x):
    if (x == 'season') or (x == 'episode'):
        return 'BRAND'
    elif x == 'collection':
        return 'COLLECTION'
    else:
        return 'PROGRAM'
    
data_implicit['type_production'] = data_implicit['type'].apply(type_production)

In [ ]:
data_implicit.head()

### Create the model

In [ ]:
data_implicit['title'] = data_implicit['title'].astype("category")
data_implicit['id_user'] = data_implicit['id_user'].astype("category")
data_implicit['id_editorial'] = data_implicit['id_editorial'].astype("category")
data_implicit['id_user_simple'] = data_implicit['id_user'].cat.codes
data_implicit['id_editorial_simple'] = data_implicit['id_editorial'].cat.codes

data_implicit.head(1)

In [ ]:
sparse_content_person = scipy.sparse.csr_matrix(
    (data_implicit['eventStrength'].astype(float), (data_implicit['id_editorial_simple'], data_implicit['id_user_simple']))
)
sparse_person_content = scipy.sparse.csr_matrix(
    (data_implicit['eventStrength'].astype(float), (data_implicit['id_user_simple'], data_implicit['id_editorial_simple']))
)

In [ ]:
%%time

model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.1, iterations=50)
alpha = 15
data_tofit = (sparse_content_person * alpha).astype('double')
model.fit(data_tofit)

In [ ]:
# # Get the trained person and content vectors. We convert them to csr matrices
# person_vecs = scipy.sparse.csr_matrix(model.user_factors)
# content_vecs = scipy.sparse.csr_matrix(model.item_factors)

### Start making recommendations

In [ ]:
id_editorial_legend = data_implicit[['id_editorial', 'id_editorial_simple', 'type_production', 'end_date']].drop_duplicates(subset=['id_editorial', 'id_editorial_simple']).sort_values(by='id_editorial_simple')

id_editorial_simple_list = id_editorial_legend['id_editorial_simple'].tolist()
id_editorial_list = id_editorial_legend['id_editorial'].tolist()
type_production_list = id_editorial_legend['type_production'].tolist()

# get array with 1 or 0 based on availability of content today
availability = id_editorial_legend['end_date'] > datetime.today().strftime('%Y-%m-%d')
availability = np.array(availability.astype(int))


id_user_legend = data_implicit[['id_user', 'id_user_simple']].drop_duplicates(subset=['id_user', 'id_user_simple']).sort_values(by='id_user_simple')
id_user_simple_list = id_user_legend['id_user_simple'].tolist()
id_user_list = id_user_legend['id_user'].tolist()

date_today = datetime.today().strftime('%Y-%m-%d')
code_version = 'v0.3'

In [ ]:
id_editorial_legend.head(3)

In [ ]:
data_implicit[data_implicit['id_editorial_simple'] == 0].head()

In [ ]:
id_user_legend.head(3)

### More Like This

In [48]:
n_similar = np.shape(model.item_factors)[0] # do it for all

content_vecs = model.item_factors

content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1)) # i.e. calculating abs. value of the vector of each item -->  |A|

def more_like_this(content_id):
    
    scores = content_vecs.dot(content_vecs[content_id,:])  / (content_norms * content_vecs[content_id,:].sum()) # i.e. calculating cosine similarity, (A.B) / (|A| x |B|) --> |B| just a constant so effectively won't need it
    
    scores =  MinMaxScaler().fit_transform(scores.reshape(-1,1))[:,0]
    
    # make zero the content no longer avilable
    scores = scores * availability
    
    similar = sorted(zip(id_editorial_list, scores[id_editorial_simple_list], type_production_list), key=lambda x: -x[1])
    
    con_id = similar[0][0]
    
    return similar[1:21], con_id # return 20, skipping first (i.e. itself)

In [49]:
%%time
# create the output for dynamoDB table
output = []

for index, i in enumerate(id_editorial_simple_list): 
    
    if availability[index] == 1: # check if content is available otherwise those recommendations will be wrong due to having multiplied scores by zero for unavailable content
    
        recs, con_id = more_like_this(i)

        output_rec = {}
        for r in recs:

            output_rec[r[0]] = {
                'score' : round(float(r[1]), 5),
                'type' : r[2],
                's_dt' : date_today,
                'c_v' : code_version
            }

        output.append([con_id, output_rec])

output_df = pd.DataFrame(output,columns=['content','recommendations'])

output_df['recommendations'] = output_df['recommendations'].apply(lambda x: json.dumps(x)) # if already a dict

# remove any duplicates; these can still occur if some content has exactly the same watch history as other and gets most similar item itself and other stuff too
output_df = output_df.drop_duplicates(subset=['content']).reset_index(drop=True)

CPU times: user 16min 55s, sys: 33.8 s, total: 17min 29s
Wall time: 1min 9s


In [50]:
output_df.head(5)

,content,recommendations
0,BBJ1003714HVOD,"{""BBJ836608HVOD"": {""score"": 0.9747964143753052..."
1,BBJ1009533HVOD,"{""BBJ359877HVOD"": {""score"": 0.9851045608520508..."
2,BBJ1009538HVOD,"{""BBJ1471735A"": {""score"": 0.9725130796432495, ..."
3,BBJ1009541HVOD,"{""BBJ1728633A"": {""score"": 0.9508183598518372, ..."
4,BBJ1009544HVOD,"{""BBJ1775457A"": {""score"": 0.9711093902587891, ..."


In [51]:
output_df[output_df['content'] == 'BBJ2788647A']

,content,recommendations
2614,BBJ2788647A,"{""BBJ2804011A"": {""score"": 0.9798159599304199, ..."


In [170]:
# save output to S3
bucket = 'bt-data-science-playground' # already created on S3
csv_buffer = StringIO()
output_df.to_csv(csv_buffer, sep='\t', index=False, quoting=csv.QUOTE_NONE)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'bt-tv-recommendation-system/output/morelikethis/test_data_MoreLikeThis.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'EE3D88D797C23EC8',
  'HostId': '15OycvYdN94vAgBHowybnwbsGCjxwQLlGOJYs9NAhiAh0V701gzy0HXyRb3EVhQ4l5zL/F0ax6k=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '15OycvYdN94vAgBHowybnwbsGCjxwQLlGOJYs9NAhiAh0V701gzy0HXyRb3EVhQ4l5zL/F0ax6k=',
   'x-amz-request-id': 'EE3D88D797C23EC8',
   'date': 'Mon, 07 Dec 2020 10:36:47 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"955c24a994f4b2c96280fae56c124436"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"955c24a994f4b2c96280fae56c124436"',
 'ServerSideEncryption': 'AES256'}

Do some tests with real titles:

In [46]:
data_implicit[data_implicit['title'] == 'An American Pickle'].head(1)

,id_user,type_asset,event_date,id_editorial,title,type,end_date,genre,rating,type_entitlement,Unnamed: 2,eventStrength,rating_n,type_production,id_user_simple,id_editorial_simple
31659383,V3009418006,film,2020-12-06 18:37:21,BBJ2788647A,An American Pickle,film,2025-12-31 23:59:00,Comedy,12,VIEW,NaN,1,2,PROGRAM,1027898,7247


In [47]:
recs, con_id = more_like_this(7247)
print(data_implicit[data_implicit['id_editorial'] == con_id]['title'].tolist()[0])
print('')
for t in recs:
    print(data_implicit[data_implicit['id_editorial'] == t[0]]['title'].tolist()[0])

An American Pickle

Spontaneous
Proxima
Rent-a-Pal
Miss Juneteenth
Lucky Grandma
Spree
London Unplugged
Stage Mother
An Accidental Studio
Z for Zachariah
Totally Under Control
3 Days with Dad
Photograph
Top End Wedding
Harpoon
The Stanford Prison Experiment
Making Waves: The Art of Cinematic Sound
Big Time Adolescence
Train To Busan Presents: Peninsula
Anomalisa


### For You

In [39]:
num_contents = np.shape(model.item_factors)[0] # do it for all

content_vecs_T = model.item_factors.T
person_vecs = model.user_factors

def for_you(person_id):
    
    # Get the interactions scores from the sparse person content matrix
    person_interactions = sparse_person_content[person_id,:].toarray()

    # Add 1 to everything, so that articles with no interaction yet become equal to 1
    person_interactions = person_interactions.reshape(-1) + 1

    # Make articles already interacted zero
    person_interactions[person_interactions > 1] = 0

    # Get dot product of person vector and all content vectors
    rec_vector = person_vecs[person_id,:].dot(content_vecs_T)

    rec_vector = MinMaxScaler().fit_transform(rec_vector.reshape(-1,1))[:,0]

    # Multiply by zero the scores of items already interacted with
    recommend_vector = person_interactions * rec_vector
    
    # make zero the content no longer avilable
    recommend_vector = recommend_vector * availability
    
    recs = sorted(zip(id_editorial_list, recommend_vector[id_editorial_simple_list], type_production_list), key=lambda x: -x[1])
    
    user_id = id_user_list[id_user_simple_list.index(person_id)]
    
    return recs[:50], user_id

In [42]:
%%time
# create the output for dynamoDB table
output = []

for i in [1039465]:#id_user_simple_list[:1000]: 
    
    recs, user_id = for_you(i)
    
    output_rec = {}
    for r in recs:
        
        output_rec[r[0]] = {
            'score' : roudn(float(r[1]), 5),
            'type' : r[2],
            's_dt' : date_today,
            'c_v' : code_version
        }
    
    output.append([user_id, output_rec])

output_df = pd.DataFrame(output,columns=['user','recommendations'])

output_df['recommendations'] = output_df['recommendations'].apply(lambda x: json.dumps(x)) # if already a dict

CPU times: user 190 ms, sys: 15 ms, total: 205 ms
Wall time: 32.9 ms


In [43]:
output_df.head(2)

,user,recommendations
0,V3578624855,"{""movida_1979"": {""score"": 0.9585089087486267, ..."


In [45]:
# save output to S3
bucket = 'bt-data-science-playground' # already created on S3
csv_buffer = StringIO()
output_df.to_csv(csv_buffer, sep='\t', index=False, quoting=csv.QUOTE_NONE)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'bt-tv-recommendation-system/output/foryou/test_data_ForYou.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '985F41DE62C160E5',
  'HostId': '9edIoUeLt5eDNVyJ/Ala5ukp08fVK1QNQwsY7/UffyaG5SNVrk3vshsH5ilpKiuvBZJ0XAXlh4o=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '9edIoUeLt5eDNVyJ/Ala5ukp08fVK1QNQwsY7/UffyaG5SNVrk3vshsH5ilpKiuvBZJ0XAXlh4o=',
   'x-amz-request-id': '985F41DE62C160E5',
   'date': 'Tue, 08 Dec 2020 12:32:22 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"8bc73315e131dac7a97fcaf8c0ffeaef"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"8bc73315e131dac7a97fcaf8c0ffeaef"',
 'ServerSideEncryption': 'AES256'}

Do some tests with real users:

In [54]:
data_implicit[data_implicit['id_user_simple'] == 0].head(10)

,id_user,type_asset,event_date,id_editorial,title,type,end_date,genre,rating,type_entitlement,Unnamed: 2,eventStrength,rating_n,type_production,id_user_simple,id_editorial_simple
7511428,V1000000000,film,2019-07-21 15:10:19,BBJ316695HVOD,Cars 2,film,2022-01-02 23:59:00,Animation,u,VIEW,NaN,1,0,PROGRAM,0,7730
20209666,V1000000000,film,2019-02-17 17:58:41,BBJ307162HVOD,Divergent,film,2021-08-16 22:59:00,Sci-Fi,12,VIEW,NaN,1,2,PROGRAM,0,7643
24463565,V1000000000,film,2019-02-17 15:12:49,movida_10027636,Avengers: Infinity War [Bonus Edition],collection,2025-12-31 23:59:00,Action,12,VIEW,NaN,1,2,COLLECTION,0,14225
25904874,V1000000000,episode,2020-06-26 20:13:11,movida_57300,MTV Cribs: Footballers Stay Home Series 1,season,2020-07-22 22:59:00,Reality,pg,VIEW,NaN,1,1,BRAND,0,18322
27451157,V1000000000,film,2020-01-10 18:08:38,BBJ339679HVOD,Guardians of the Galaxy,film,2022-01-02 23:59:00,Action,12,VIEW,NaN,1,2,PROGRAM,0,7848
31603714,V1000000000,film,2019-03-22 16:19:36,movida_10027636,Avengers: Infinity War [Bonus Edition],collection,2025-12-31 23:59:00,Action,12,VIEW,NaN,1,2,COLLECTION,0,14225
31603758,V1000000000,film,2019-03-22 15:49:03,movida_10027636,Avengers: Infinity War [Bonus Edition],collection,2025-12-31 23:59:00,Action,12,VIEW,NaN,1,2,COLLECTION,0,14225
34804100,V1000000000,film,2018-08-04 11:03:45,BBJ314269HVOD,Saving Private Ryan,film,2025-12-31 23:59:00,Action,15,VIEW,NaN,1,3,PROGRAM,0,7654
43925453,V1000000000,episode,2019-10-02 21:31:11,movida_10070789,Man City v Zagreb,collection,2021-06-30 22:59:00,Football,u,VIEW,NaN,1,0,COLLECTION,0,16206
44235449,V1000000000,film,2020-01-19 13:43:13,BBJ1060095HVOD,The Hobbit: The Desolation of Smaug,film,2022-01-02 23:59:00,Fantasy,12,VIEW,NaN,1,2,PROGRAM,0,463


In [55]:
recs, con_id = for_you(0)

for t in recs:
     print(data_implicit[data_implicit['id_editorial'] == t[0]]['title'].tolist()[0])

The Legend of Tarzan
Avengers: Endgame [Bonus Edition]
Deadpool 2 [Bonus Edition]
Deadpool 2
Black Panther
Venom
Thor: Ragnarok [Bonus Edition]
Ant-Man and the Wasp
Solo: A Star Wars Story [Bonus Edition]
Star Wars: The Last Jedi [Bonus Edition]
SNATCH Series 1
Ready Player One
Game of Thrones Series 1
The Harry Potter Complete Collection
Ready Player One [Bonus Edition]
Guardians of the Galaxy Vol. 2 [Bonus Edition]
Bohemian Rhapsody [Bonus Edition]
A Star Is Born
John Wick: Chapter 3 - Parabellum
Pacific Rim: Uprising
Bumblebee
Spider-Man: Homecoming
Fantastic Beasts And Where To Find Them
Shazam!
Johnny English Strikes Again
Star Wars: The Rise of Skywalker [Bonus Edition]
Avengers Assemble
Star Wars: The Force Awakens [Bonus Edition]
Justice League
The House With A Clock In Its Walls
Incredibles 2 [Bonus Edition]
UEFA Champions League 2018/19
Jumanji: The Next Level
Kingsman: The Golden Circle
Ralph Breaks the Internet
Rogue One: A Star Wars Story [Bonus Edition]
John Wick: Chapter

In [34]:
data_implicit[data_implicit['id_user'] == 'V2283739102'].head(1) # Sam's

,id_user,type_asset,event_date,id_editorial,title,type,end_date,genre,rating,type_entitlement,Unnamed: 2,eventStrength,rating_n,type_production,id_user_simple,id_editorial_simple
102318,V2283739102,episode,2019-11-11 16:13:39,movida_29422,Sesame Street ABC's Series 1,season,2021-10-27 22:59:00,2 - 5 Years,u,VIEW,NaN,1,0,BRAND,45140,17941


In [56]:
recs, con_id = for_you(45140)

for t in recs:
     print(data_implicit[data_implicit['id_editorial'] == t[0]]['title'].tolist()[0])

Wilder v Fury II
Premier League World 2020/21
UEFA Champions League 2019/20
Frozen II [Bonus Edition]
Ben 10: Alien Worlds Series 3
Mighty Mike Series 1
Scooby-Doo and Guess Who? Series 1
Teen Titans Go! Series 1
WWE Raw
WWE SmackDown
Scooby-Doo! Mystery Incorporated Series 2
NinjaGo Series 1
The Amazing World of Gumball Series 5
Angry Birds Blues Series 1
Be Cool, Scooby-Doo! Series 1
Taffy Series 1
Toy Story 4 [Bonus Edition]
Greavsie
The Scooby Doo Show Series 1
Onward
Tom and Jerry Tales Series 1
Scooby Doo Where Are You! Series 2
Jumanji: The Next Level
Dorothy and the Wizard of Oz Series 1
Peppa Pig Series 2
Aladdin
Teen Titans Go!
Uncle Grandpa Series 1
Paw Patrol Series 5
ESPN Films: 30 for 30
Ben & Holly's Little Kingdom Series 1
Fireman Sam Series 9
Toy Story 4
Wilder v Fury Highlights
Joker
Top 10 UCL Goals Messi
Scooby-Doo!
ZeZe Zebra Nursery Rhymes
Upgrade
The Batman Series 1
The Looney Tunes Show Series 1
Super Healthy Monsters Series 1
UEFA Champions League 2020/21
XLR8


In [40]:
data_implicit[data_implicit['id_user'] == 'V3578624855'].head(1) # Peter's

,id_user,type_asset,event_date,id_editorial,title,type,end_date,genre,rating,type_entitlement,Unnamed: 2,eventStrength,rating_n,type_production,id_user_simple,id_editorial_simple
68100,V3578624855,episode,2018-11-01 09:30:37,movida_10018585,SpongeBob SquarePants Series 11,season,2020-02-03 04:30:00,6 Years +,u,VIEW,NaN,1,0,BRAND,1039465,13711


In [41]:
recs, con_id = for_you(1039465)

for t in recs:
     print(data_implicit[data_implicit['id_editorial'] == t[0]]['title'].tolist()[0])

Game of Thrones Series 4
TUF: Season 28
IT
Daddy's Home 2
The Harry Potter Complete Collection
Ready Player One [Bonus Edition]
Deadpool 2 [Bonus Edition]
Young Sheldon Series 1
Kingsman: The Golden Circle
Shazam!
UFC Ultimate Knockouts
Home Alone
Elf
Mean Girls
Home Alone 2: Lost in New York
UFC 25 Greatest Fights
Suicide Squad [Bonus Edition]
Henry Danger Series 5
Lost Series 1
The Office (US) Series 7
Goosebumps 2: Haunted Halloween
Pokemon Detective Pikachu
The Simpsons Movie
IT Chapter Two
UFC 244
Harry Potter and the Philosopher's Stone
Harry Potter and the Chamber of Secrets
Hocus Pocus
Justice League
Grease
UFC Top 10s
Fantastic Beasts And Where To Find Them
Apple & Onion Series 1
We Bare Bears Series 2
The Flash Series 2
Ant-Man and the Wasp
Wonder Woman
Bumblebee
Avengers: Age of Ultron
UFC 230
Tim Burton's The Nightmare Before Christmas
Creed II
Captain America: Civil War [Bonus Edition]
UFC 235
The Lord of the Rings Trilogy
Instant Family
The Muppet Christmas Carol
Good Boy